In [31]:
import pandas as pd
import swifter # Looks unused but actually is through Monkey Patching -- don't delete
import matplotlib.pyplot as plt
import re
import geopandas as gpd
import numpy as np
from src.utils.data_utils import *
from src.utils.general_utils import *
from tqdm import tqdm
from prettytable import PrettyTable,MARKDOWN
from ipywidgets import interact
import ast
from itertools import combinations
import seaborn as sns
import networkx as nx
import json
from sklearn.linear_model import RANSACRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib.colors import LogNorm
from matplotlib.ticker import FuncFormatter

color_palette = {
    "Persian indigo": "#332288",
    "Baby blue": "#88ccee",
    "Harvest gold": "#e69f00",
    "Keppel": "#44aa99",
    "Maize": "#f0e442",
    "Tawny": "#d55e00",
    "Murrey": "#882255",
    "Shamrock green": "#009e73"
}

#df = genre_counts_df.dropna()

# Choisir une palette personnalisée
custom_palette = [color_palette["Persian indigo"], color_palette["Baby blue"], color_palette["Harvest gold"], color_palette["Keppel"]]


N = 20

pd.options.mode.copy_on_write = True

name_csv = 'optimal_matches.csv'

In [33]:


def loaf_DF_Quentin(name_csv):

    Dataframe_csv = pd.read_csv(name_csv)
    return Dataframe_csv

def extract_genres_cleaned(genre_str):
    # Convertir la chaîne en dictionnaire
    genre_dict = ast.literal_eval(genre_str)
    # Extraire les genres intelligibles (valeurs uniquement)
    return [v for v in genre_dict.values() if v != "Comedy"]

def generate_combinations(genres):
    if len(genres) < 2:
        return []  # Pas de combinaisons possibles si moins de 2 genres
    return list(combinations(genres, 2))

def extract_genres(genre_data):
    try:
        # Si c'est une chaîne, convertir en dictionnaire
        if isinstance(genre_data, str):
            genre_dict = json.loads(genre_data.replace("'", "\""))
        elif isinstance(genre_data, dict):
            genre_dict = genre_data
        else:
            return []  # Si ce n'est ni une chaîne ni un dictionnaire, retourner une liste vide

        # Retourner uniquement les genres intelligibles
        return [v for v in genre_dict.values() if v != "Comedy"]
    except (json.JSONDecodeError, ValueError):
        # Si la chaîne est invalide ou ne peut pas être convertie
        return []

def beter_csv_quentin (name_csv):
    df_new_combination_genre = pd.DataFrame()
    df_old = loaf_DF_Quentin(name_csv)
 
    df_new_combination_genre['filtered_genres'] = df_old['genres'].apply(extract_genres)
    df_new_combination_genre ['audienceScore'] = df_old['audienceScore']
    df_new_combination_genre ['tomatoMeter'] = df_old['tomatoMeter']
    
    # le_bon_DF = pd.DataFrame(df_new_combination_genre['filtered_genres'])
    le_bon_DF = df_new_combination_genre
    return le_bon_DF

def DF_with_unic_genre_quentin (name_csv):
    df_new_combination_genre = beter_csv_quentin(name_csv)
    GEnres_par_film = pd.DataFrame(df_new_combination_genre['filtered_genres'])
    all_genres = pd.Series([genre for genres_list in GEnres_par_film['filtered_genres'] for genre in genres_list])

    # Étape 2 : Compter les occurrences de chaque genre
    genre_counts = all_genres.value_counts()
    
    genre_counts_df = pd.DataFrame({'Genre': genre_counts.index, 'Count': genre_counts.values})

    genre_counts_df ['audienceScore'] = df_new_combination_genre['audienceScore']
    genre_counts_df ['tomatoMeter'] = df_new_combination_genre['tomatoMeter']
    genre_counts_df ['moyenne_score'] = (genre_counts_df['audienceScore'] + genre_counts_df['tomatoMeter']) / 2
    return genre_counts_df


def DF_pour_paires_de_genre_quentin(name_csv):
    df_new_combination_genre = DF_with_unic_genre_quentin (name_csv)
    optimal_matches = loaf_DF_Quentin(name_csv)
    df_new_combination_genre['genre_combinations'] = df_new_combination_genre['filtered_genres'].apply(generate_combinations)
    df_new_combination_genre['audienceScore'] = optimal_matches['audienceScore']
    df_new_combination_genre['tomatoMeter'] = optimal_matches['tomatoMeter']

    rows = []
    for _, row in df_new_combination_genre.iterrows():
        for combo in row['genre_combinations']:
            rows.append({
                'combination': combo,
                'audienceScore': row['audienceScore'],
                'tomatoMeter': row['tomatoMeter']
            })

    # Créer un nouveau dataframe
    combination_genders_df = pd.DataFrame(rows)
    return combination_genders_df


def clean_genres(genre_string):
    # Nettoie les caractères indésirables comme les guillemets et les parenthèses
    return [re.sub(r"[\'\(\)\[\]]", "", genre).strip() for genre in genre_string.split(', ')]

def clean_combination_genr_quentin (name_csv):
    combos_df = DF_pour_paires_de_genre_quentin(name_csv)
    for row in combos_df.itertuples():
        genre1, genre2 = sorted(clean_genres(str(row.combination)))
        

    genres = []
    genres2 = []
    genre1_arr = []
    genre2_arr = []
    for combo in combos_df['combination']:
        combo_clean = clean_genres(str(combo))
        genre1 = combo_clean [0]
        genre2 = combo_clean [1]
        genre1_arr.append(genre1)
        genre2_arr.append(genre2)
        genres = genre1 + ',' + genre2
        genres2.append(genres)
        if genre1 not in genres:
            genres.append(genre1)
        if genre2 not in genres:
            genres.append(genre2)

    nouveau_DF = pd.DataFrame(combos_df)

    nouveau_DF ['combinations_bis'] = genres2
    nouveau_DF ['genre1'] = genre1_arr
    nouveau_DF ['genre2'] = genre2_arr
    
    return nouveau_DF




# Fonction pour créer des combinaisons triées
def create_sorted_combinations(genres):
    if len(genres) < 2:
        return []  # Pas de combinaisons possibles
    return [tuple(sorted(comb)) for comb in combinations(genres, 2)]



def normalize_genres(genres):
    normalized = []
    for genre in genres:
        # Si le genre contient un '/', on le sépare
        if '/' in genre:
            normalized.extend(genre.split('/'))
        else:
            # Ne garder que le premier mot d'un genre
            normalized.append(genre.split()[0])
    return list(set(normalized))  # Supprimer les doublons

def first_combination_genre (csv_name):
    df_new_combination_genre = beter_csv_quentin(csv_name)
    
    # Appliquer la fonction sur la colonne "filtered_genres"
    df_new_combination_genre['combinations'] = df_new_combination_genre['filtered_genres'].apply(create_sorted_combinations)
    # Développer les combinaisons en lignes séparées
    exploded_df = df_new_combination_genre.explode('combinations')

    # Supprimer les lignes avec des valeurs NaN (si certaines combinaisons étaient vides)
    exploded_df = exploded_df.dropna(subset=['combinations'])
    
    # Calculer les statistiques pour chaque combinaison unique
    final_df = (
        exploded_df.groupby('combinations')
        .agg(
            nombredoccurrence=('combinations', 'size'),
            audienceScoremoyenne=('audienceScore', 'mean'),
            tomatoMetermoyen=('tomatoMeter', 'mean')
        )
        .reset_index()
    )



    df_new_combination_genre['normalized_genres'] = df_new_combination_genre['filtered_genres'].apply(normalize_genres)

    df_new_combination_genre['combinations'] = df_new_combination_genre['normalized_genres'].apply(create_sorted_combinations)

    # Développer les combinaisons en lignes séparées
    exploded_df = df_new_combination_genre.explode('combinations')

    # Supprimer les lignes avec des valeurs NaN
    exploded_df = exploded_df.dropna(subset=['combinations'])

    # Calculer les statistiques pour chaque combinaison unique
    final_df = (
        exploded_df.groupby('combinations')
        .agg(
            nombredoccurrence=('combinations', 'size'),
            audienceScoremoyenne=('audienceScore', 'mean'),
            tomatoMetermoyen=('tomatoMeter', 'mean')
        )
        .reset_index()
    )

    final_df = final_df.sort_values(by='nombredoccurrence', ascending=False)

    # Réinitialiser les index après le tri (optionnel)
    final_df = final_df.reset_index(drop=True)
    return final_df







In [ ]:
#others
def ash_quentin (csv_name):
    final_df = first_combination_genre (csv_name)
    final_df['combinations'] = final_df['combinations'].astype(str)
    return final_df

In [ ]:
#saves

def save_final (csv_name):
    final_df = first_combination_genre (csv_name)
    final_df.to_csv('le_csv_genre_combination.csv', index=False)


In [19]:
le_bon_df = beter_csv_quentin(name_csv)
le_bon_df

,filtered_genres,audienceScore,tomatoMeter
0,"[Romantic comedy, Romance Film, Slice of life ...",74.0,96.0
1,"[Science Fiction, Action, Adventure]",79.0,90.0
2,"[Parody, Sex comedy, Horror, Science Fiction, ...",85.0,79.0
3,"[Romance Film, Drama]",90.0,86.0
4,"[Romantic comedy, Romantic drama, Romance Film...",59.0,17.0
...,...,...,...
5998,"[Short Film, Romance Film, Family Film, Animat...",32.0,NaN
5999,[Romance Film],79.0,NaN
6000,[Travel],NaN,NaN
6001,[Musical],NaN,NaN


In [20]:
unic_gen_df = DF_with_unic_genre_quentin(name_csv)
unic_gen_df

,Genre,Count,audienceScore,tomatoMeter,moyenne_score
0,Drama,1647,74.0,96.0,85.0
1,Romance Film,1497,79.0,90.0,84.5
2,Romantic comedy,1162,85.0,79.0,82.0
3,Indie,1003,90.0,86.0,88.0
4,Black-and-white,984,59.0,17.0,38.0
...,...,...,...,...,...
257,Animated Musical,1,72.0,62.0,67.0
258,Interpersonal Relationships,1,77.0,85.0,81.0
259,Star vehicle,1,50.0,73.0,61.5
260,Erotic Drama,1,56.0,27.0,41.5


In [24]:
# def de tout ce qui nous sert à plot des graphes. attention, ceux-ci prennent des DataFrames, pas des noms de csv.

def plot_top20_frequence (dataframe):
    top_20_genres = dataframe['Count'].head(N)
    top_20_genres_genre = dataframe['Genre'].head(N)

    # Étape 3 : Créer un histogramme
    color_palette

    norm = plt.Normalize(vmin=top_20_genres.min(), vmax=top_20_genres.max())
    cmap = plt.get_cmap('plasma')  # Choisir le colormap (vous pouvez changer le nom ici)

    # Étape 4 : Créer un histogramme avec couleur dynamique selon la valeur
    fig, ax = plt.subplots(figsize=(12, 6))

    # Créer les barres avec les couleurs correspondant à la valeur
    bars = plt.bar(top_20_genres_genre, top_20_genres.values, color=cmap(norm(top_20_genres.values)))

    # Ajouter une barre de couleur pour la légende
    cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), ax=plt.gca())
    cbar.set_label('Occurrences', rotation=270, labelpad=20)
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, yval, str(int(yval)), ha='center', va='bottom', rotation=90, fontsize=10)


    # Titres et axes
    plt.title('Top 20 Most Frequent Genres', fontsize=16)
    ax.set_xlabel('Genres', fontsize=14)
    ax.set_ylabel('Occurrences', fontsize=14)
    ax.set_xticklabels(top_20_genres_genre, rotation=75, fontsize=12)
    plt.tight_layout()  # Ajuster pour éviter le chevauchement


    # Enregistrer le graphique en JPG et SVG
    plot_folder_path = './LesPlots/'
    plt.savefig("histogramme_unic_genres_compts.jpg", format='jpg', dpi=300)  # Enregistrer en JPG avec une bonne résolution
    plt.savefig( "histogramme_unic_genres_compts.svg", format='svg')
    plt.show()





def plot_fig4_quentin (genre_counts_df):
    # Assuming genre_counts_df is already loaded and cleaned
    
    # genre_counts_df = DF_with_unic_genre_quentin(csv_name)
    df = genre_counts_df.dropna()

    # Initialize the 2x2 panel
    fig, axs = plt.subplots(2, 2, figsize=(16, 12))

    #####################
    # Plot 1: RANSAC Mean Score vs Count
    X = df['moyenne_score'].values.reshape(-1, 1)
    y = df['Count'].values

    ransac = RANSACRegressor()
    ransac.fit(X, y)

    y_pred = ransac.predict(X)

    axs[0, 0].scatter(df['moyenne_score'], df['Count'], color='blue', label='Data')
    axs[0, 0].plot(df['moyenne_score'], y_pred, color='red', label='Adjusted Line (RANSAC)')
    axs[0, 0].set_title('RANSAC: Mean Score vs Count', fontsize=14)
    axs[0, 0].set_xlabel('Mean Score', fontsize=12)
    axs[0, 0].set_ylabel('Genre Count', fontsize=12)
    axs[0, 0].legend()

    #####################
    # Plot 2: Random Forest Count vs Mean Score
    X = df['Count'].values.reshape(-1, 1)
    y = df['moyenne_score'].values

    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X, y)

    y_pred_rf = rf.predict(X)

    axs[0, 1].scatter(df['Count'], df['moyenne_score'], color='blue', label='Data')
    axs[0, 1].plot(df['Count'], y_pred_rf, color='red', label='Fit by Random Forest')
    axs[0, 1].set_title('Random Forest: Count vs Mean Score', fontsize=14)
    axs[0, 1].set_xlabel('Genre Count', fontsize=12)
    axs[0, 1].set_ylabel('Mean Score', fontsize=12)
    axs[0, 1].legend()

    #####################
    # Plot 3: Random Forest (Count < 40)
    filtered_df = df[df['Count'] < 40]

    X = filtered_df['Count'].values.reshape(-1, 1)
    y = filtered_df['moyenne_score'].values

    rf = RandomForestRegressor(n_estimators=500, random_state=42)
    rf.fit(X, y)

    y_pred_rf = rf.predict(X)

    axs[1, 0].scatter(filtered_df['Count'], filtered_df['moyenne_score'], color='blue', label='Data (Count < 40)')
    axs[1, 0].plot(filtered_df['Count'], y_pred_rf, color='red', label='Fit by Random Forest')
    axs[1, 0].set_title('Random Forest: Genre Count < 40', fontsize=14)
    axs[1, 0].set_xlabel('Genre Count', fontsize=12)
    axs[1, 0].set_ylabel('Mean Score', fontsize=12)
    axs[1, 0].legend()

    #####################
    # Plot 4: RANSAC (Count < 40)
    X = filtered_df['moyenne_score'].values.reshape(-1, 1)
    y = filtered_df['Count'].values

    ransac = RANSACRegressor()
    ransac.fit(X, y)

    y_pred = ransac.predict(X)

    axs[1, 1].scatter(filtered_df['moyenne_score'], filtered_df['Count'], color='blue', label='Data')
    axs[1, 1].plot(filtered_df['moyenne_score'], y_pred, color='red', label='Adjusted Line (RANSAC)')
    axs[1, 1].set_title('RANSAC: Count < 40', fontsize=14)
    axs[1, 1].set_xlabel('Mean Score', fontsize=12)
    axs[1, 1].set_ylabel('Genre Count', fontsize=12)
    axs[1, 1].legend()

    # Adjust layout and save the figures
    plt.tight_layout()
    plt.savefig('plots_panel.png', format='png')
    plt.savefig('plots_panel.svg', format='svg')
    plt.show()




def plot_score_best20_quentin (dataframe):
    # Trier les genres par Count
    df = dataframe.dropna()
    top_20_genres = df.sort_values(by='Count', ascending=False).head(20)

    # Créer une palette de couleurs en fonction du score moyen
    cmap = cm.plasma  # Colormap de Matplotlib
    norm = mcolors.Normalize(vmin=top_20_genres['moyenne_score'].min(), vmax=top_20_genres['moyenne_score'].max())

    # Créer le plot
    fig, ax = plt.subplots(figsize=(14, 8))  # Définir explicitement les axes
    bars = ax.bar(
        top_20_genres['Genre'], 
        top_20_genres['Count'],  # Utiliser Count comme hauteur des barres
        color=cmap(norm(top_20_genres['moyenne_score']))  # Couleur basée sur moyenne_score
    )
    print(top_20_genres['Count'])
    # Ajouter des annotations pour les hauteurs des barres
    i=0
    arr_ct = np.array(top_20_genres['moyenne_score'])
    for bar, count in zip(bars, top_20_genres['Count']):
        indexxx =  top_20_genres[top_20_genres['Count'] == count].index[0]
        
        # print(count)
        # print(arr_ct)
        ax.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height(),
            #f'{count}',#arr_ct[indexxx]
            f'{arr_ct[i]}',
            ha='center',
            va='bottom',
            fontsize=10
        )
        i+=1

    # Ajouter une Colorbar explicitement liée aux axes
    sm = cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])  # Nécessaire pour ScalarMappable
    cbar = plt.colorbar(sm, ax=ax)  # Associer explicitement la colorbar à l'axe
    cbar.set_label('Average Score [%]', fontsize=12)

    # Ajuster les labels et le titre
    ax.set_title("Top 20 Genres by Count (Color = Average Score)", fontsize=16)
    ax.set_xlabel("Genre", fontsize=14)
    ax.set_ylabel("Count", fontsize=14)
    ax.set_xticklabels(top_20_genres['Genre'], rotation=75, ha='right', fontsize=12)

    # Afficher et sauvegarder
    plt.tight_layout()
    plt.savefig('top_20_genres_by_count.png', format='png')
    plt.savefig('top_20_genres_by_count.svg', format='svg')
    plt.show()

    mntot = np.mean(np.array(df['moyenne_score']))

    mn = np.mean(arr_ct)
    print('The average score for the 20 most represented genres in comedy is:', mn)
    print('The average score for the all genres in comedy is:', mntot)


def plot_giga_map (nouveau_DF):
        # Étape 1 : Calculer les poids des liens
    edge_weights = nouveau_DF.groupby(["genre1", "genre2"]).size().reset_index(name="weight")

    # Étape 2 : Calculer les poids des nœuds
    node_weights = {}
    for _, row in edge_weights.iterrows():
        g1, g2, weight = row["genre1"], row["genre2"], row["weight"]
        node_weights[g1] = node_weights.get(g1, 0) + weight
        node_weights[g2] = node_weights.get(g2, 0) + weight

    # Ajouter le nœud central "Comedy"
    node_weights["Comedy"] = 6003

    # Étape 3 : Créer le graphe
    G = nx.Graph()

    # Ajouter les nœuds et leurs poids
    for node, weight in node_weights.items():
        G.add_node(node, size=weight)

    # Ajouter les liens et leurs poids
    for _, row in edge_weights.iterrows():
        G.add_edge(row["genre1"], row["genre2"], weight=row["weight"])

    # Ajouter les liens pour le nœud central "Comedy"
    for node in node_weights.keys():
        if node != "Comedy":
            G.add_edge("Comedy", node, weight=0)  # Liens avec poids 0 (ou aucun lien si besoin)

    # Étape 4 : Positionner les nœuds
    # Fixer "Comedy" au centre
    delta = 0.5
    pos = nx.spring_layout(G, seed=42)  # Disposition forcée
    pos["Comedy"] = [0, 0]  # Forcer la position de "Comedy" au centre
    for node in pos:
        if node != "Comedy":
            pos[node] = pos[node]* 150 + [np.sign(pos[node][0]) * delta , np.sign(pos[node][1]) * delta ]  # Espacer les autres nœuds pour plus de lisibilité

    # Étape 5 : Appliquer une échelle logarithmique à la taille des nœuds
    node_sizes = [G.nodes[node]["size"] for node in G.nodes]
    log_node_sizes = [np.log(size + 1) * 2000 for size in node_sizes]  # Taille logarithmique des nœuds
    node_colors = node_sizes  # Couleurs linéaires en fonction des poids des nœuds

    # Étape 6 : Visualiser le graphe
    plt.figure(figsize=(80, 70))
    nx.draw_networkx_nodes(G, pos, node_size=log_node_sizes, 
                        node_color=node_colors, cmap="viridis", alpha=0.8)
    nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5, edge_color="gray")
    nx.draw_networkx_labels(G, pos, font_size=10, font_weight="bold")

    # Ajouter une barre de couleur pour représenter les poids (linéaire)
    sm = plt.cm.ScalarMappable(cmap="plasma", norm=plt.Normalize(vmin=min(node_sizes), vmax=max(node_sizes)))
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=plt.gca(), fraction=0.03, pad=0.04)
    cbar.set_label("Poids des nœuds (linéaire)", fontsize=12)

    # Ajouter un titre
    plt.title("Graphique des genres avec 'Comedy' comme nœud central", fontsize=16)
    plt.axis("off")
    plt.tight_layout()

    # Sauvegarder et afficher
    plt.savefig("genre_graph_with_comedy_log_nodes.png", dpi=300, bbox_inches="tight")
    plt.savefig("genre_graph_with_comedy_log_nodes.svg", format="svg", bbox_inches="tight")
    plt.show()
    



#here df = first_combination_genre(csv_name)

def graph_frequence_most_commun_copple_genre_quentin (df):
    # Sélectionner les 15 combinaisons les plus fréquentes
    final_df = df
    final_df['combinations'] = final_df['combinations'].astype(str)
    top_15_combinations = final_df.head(15)

    # Créer l'histogramme
    # Définir la taille et le style de la figure
    plt.figure(figsize=(14, 8))
    sns.set_style("whitegrid")  # Appliquer un style esthétique
    sns.set_context("talk", font_scale=1.2)  # Ajuster le contexte global et la taille de la police


    # Création du barplot
    ax = sns.barplot(
        data=top_15_combinations,
        x='combinations',
        y='nombredoccurrence',
        palette=custom_palette
    )


    # Ajouter les valeurs sur chaque barre
    for p in ax.patches:
        ax.annotate(f'{p.get_height():.0f}', 
                    (p.get_x() + p.get_width() / 2., p.get_height()), 
                    ha='center', va='center', 
                    fontsize=14, color='black', 
                    xytext=(0, 5), textcoords='offset points')

    # Ajuster les labels
    plt.xticks(rotation=65, ha='right', fontsize=14)
    plt.xlabel('Combinations of Genres', fontsize=14)
    plt.ylabel('Number of Occurrences', fontsize=14)
    plt.title('Top 15 Most Common Genre Combinations', fontsize=16)
    plt.tight_layout()

    plt.savefig( "top_genre_combinations.png", dpi=300, bbox_inches='tight')
    # Afficher le graphique
    plt.show()



N = 20

# Étape 1 : Nettoyer les genres et extraire tous les genres uniques dans les combinaisons
def clean_genres(genre_string):
    # Nettoie les caractères indésirables comme les guillemets et les parenthèses
    return [re.sub(r"[\'\(\)\[\]]", "", genre).strip() for genre in genre_string.split(', ')]



#here df = ash_quentin (csv_name)
def plot_fig_couple_frequences (df):

    # Extraire et nettoyer les genres uniques
    genres = []
    for combo in df['combinations']:
        combo_clean = clean_genres(combo)
        genre1 = combo_clean [0]
        genre2 = combo_clean [1]

        if genre1 not in genres:
            genres.append(genre1)
        if genre2 not in genres:
            genres.append(genre2)

        if len(genres) > N:
            break

    top_20_genres = genres[:N]  # Limiter à 20 genres (ou tout autre nombre N)


    # Étape 2 : Créer une matrice vide pour les genres
    genre_matrix = pd.DataFrame(0, index=top_20_genres, columns=top_20_genres)

    # Étape 3 : Remplir la matrice avec les fréquences des combinaisons
    for combo, count in zip(df['combinations'], df['nombredoccurrence']):
        genres_in_combo = clean_genres(combo)
        
        # Ne conserver que les genres qui figurent dans le top_20_genres
        genres_in_combo = [genre for genre in genres_in_combo if genre in top_20_genres]

        # Vérifier que genres_in_combo contient plus d'un genre, sinon ignorer cette combinaison
        if len(genres_in_combo) > 1:
            # Créer une matrice symétrique des combinaisons (genre1, genre2)
            for i in range(len(genres_in_combo)):
                for j in range(i + 1, len(genres_in_combo)):
                    genre1, genre2 = sorted([genres_in_combo[i], genres_in_combo[j]])

                    # Vérifier si la combinaison (genre1, genre2) existe, sinon (genre2, genre1)
                    if genre_matrix.loc[genre1, genre2] == 0:
                        genre_matrix.loc[genre1, genre2] = count
                    else:
                        genre_matrix.loc[genre2, genre1] = count

                    # Assurer la symétrie (genre1, genre2) == (genre2, genre1)
                    genre_matrix.loc[genre2, genre1] = genre_matrix.loc[genre1, genre2]  # Assurer la symétrie


    genre_matrix_filled = genre_matrix.fillna(0)

    # Créer un masque où les valeurs sont égales à 0
    zero_mask = genre_matrix_filled == 0

    # Appliquer une échelle logarithmique, mais ignorer les 0 dans l'échelle des couleurs
    plt.figure(figsize=(12, 10))

    # Créer la heatmap avec une échelle logarithmique, mais afficher les 0 en blanc
    sns.heatmap(genre_matrix_filled, annot=True, cmap='plasma', linewidths=0.5, fmt='.0f', 
                cbar_kws={'label': 'Occurrences'},
                norm=LogNorm(vmin=np.nanmin(genre_matrix.values[~np.isnan(genre_matrix.values)]+1), 
                            vmax=np.nanmax(genre_matrix.values[~np.isnan(genre_matrix.values)])),  # Appliquer LogNorm uniquement aux valeurs non-NaN
                mask=zero_mask,  # Masquer les 0
                annot_kws={'size': 8})

    # Ajouter un titre et ajuster les étiquettes
    plt.title('best sub-genres combination of comedie movies by occurrences')
    plt.xticks(rotation=90, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig( "heatmap_nombre_apparition_combinaisons_genre.png", dpi=300, bbox_inches='tight')
    plt.show()

def clean_genres(genre_string):
    # Nettoie les caractères indésirables comme les guillemets et les parenthèses
    return [re.sub(r"[\'\(\)\[\]]", "", genre).strip() for genre in genre_string.split(', ')]




#here df = ash_quentin (csv_name)
def plot_graph_best_score_gender_combination_quentin (df):
    df_sort_public_score = df.sort_values(by='audienceScoremoyenne', ascending=False)

    # Réinitialiser les index après le tri (optionnel)
    df_sort_public_score = df_sort_public_score.reset_index(drop=True)
    df_sort_public_score
    df_sort_public_score = df_sort_public_score.dropna(subset=['audienceScoremoyenne', 'tomatoMetermoyen'])
    df_sort_public_score = df_sort_public_score.reset_index(drop=True)

        # Liste de genres à supprimer car pas ouf
    genres_to_remove = ['Slasher', 'New', 'Pre-Code', 'Humour', 'History', 'Hybrid', 'Existentialism', 'Addiction', 'Gay', 'Costume', 'Childhood', 'Melodrama', 'Neo-noir', 'Computer', 'Fairy', 'Kafkaesque', 'Future', 'Educational' ]

    # Fonction pour vérifier si l'un des genres à supprimer est dans la combinaison
    def contains_genre(combo, genres_to_remove):
        return any(genre in combo for genre in genres_to_remove)

    # Supprimer les lignes contenant les genres de la liste dans 'combinations'
    df_sort_public_score = df_sort_public_score[~df_sort_public_score['combinations'].apply(lambda x: contains_genre(x, genres_to_remove))]

    df_sort_public_score = df_sort_public_score.reset_index(drop=True)
    


        # Extraire et nettoyer les genres uniques
    genres = []
    for combo in df_sort_public_score['combinations']:
        combo_clean = clean_genres(combo)
        genre1 = combo_clean [0]
        genre2 = combo_clean [1]

        if genre1 not in genres:
            genres.append(genre1)
        if genre2 not in genres:
            genres.append(genre2)

        if len(genres) > N:
            break


    top_20_genres = genres[:N]  # Limiter à 20 genres (ou tout autre nombre N)


    # Étape 2 : Créer une matrice vide pour les genres
    genre_matrix = pd.DataFrame(0, index=top_20_genres, columns=top_20_genres)

    # Étape 3 : Remplir la matrice avec les fréquences des combinaisons
    for combo, count in zip(df['combinations'], df['audienceScoremoyenne']):
        genres_in_combo = clean_genres(combo)
        
        # Ne conserver que les genres qui figurent dans le top_20_genres
        genres_in_combo = [genre for genre in genres_in_combo if genre in top_20_genres]

        # Vérifier que genres_in_combo contient plus d'un genre, sinon ignorer cette combinaison
        if len(genres_in_combo) > 1:
            # Créer une matrice symétrique des combinaisons (genre1, genre2)
            for i in range(len(genres_in_combo)):
                for j in range(i + 1, len(genres_in_combo)):
                    genre1, genre2 = sorted([genres_in_combo[i], genres_in_combo[j]])

                    # Vérifier si la combinaison (genre1, genre2) existe, sinon (genre2, genre1)
                    if genre_matrix.loc[genre1, genre2] == 0:
                        genre_matrix.loc[genre1, genre2] = count
                    else:
                        genre_matrix.loc[genre2, genre1] = count

                    # Assurer la symétrie (genre1, genre2) == (genre2, genre1)
                    genre_matrix.loc[genre2, genre1] = genre_matrix.loc[genre1, genre2]  # Assurer la symétrie

        # Remplacer les NaN par 0 pour les afficher en blanc
    genre_matrix_filled = genre_matrix.fillna(0)

    # Créer un masque où les valeurs sont égales à 0
    zero_mask = genre_matrix_filled == 0

    # Trouver le minimum des valeurs non nulles
    arr = genre_matrix.values[~np.isnan(genre_matrix.values)]
    vminim = np.min(arr[arr > 0])  # Minimum non nul
    vminim = int(vminim)  # Convertir en entier pour l'affichage
    vmaxim = int(np.nanmax(arr))  # Maximum des valeurs

    print(f"Minimum (non nul) pour LogNorm: {vminim}")
    print(f"Maximum pour LogNorm: {vmaxim}")

    # Créer la heatmap avec une échelle logarithmique
    plt.figure(figsize=(12, 10))

    ax = sns.heatmap(
        genre_matrix_filled, 
        cmap='plasma', 
        annot=True, 
        fmt=".0f", 
        cbar_kws={'label': 'Occurrences'},
        norm=LogNorm(vmin=vminim, vmax=vmaxim),  # Appliquer LogNorm
        mask=zero_mask,  # Masquer les 0
        annot_kws={'size': 8}
    )

    # Récupérer la colorbar associée
    cbar = ax.collections[0].colorbar

    # Appliquer un format personnalisé pour afficher les ticks en entier
    cbar.ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{int(x)}"))

    # Ajouter un titre et ajuster les étiquettes
    plt.title('Heatmap des meilleures combinaisons de sous-genres (Score Audience)')
    plt.xticks(rotation=90, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()

    # Sauvegarder l'image
    path_plots_save = "./LesPlots/"  # Remplacez par le chemin où vous voulez sauvegarder
    plt.savefig(path_plots_save + "heatmap_best_Audiance_scoreby_subgenre_combination.png", dpi=300, bbox_inches='tight')

    # Afficher la heatmap
    plt.show()

    




#here df = ash_quentin (csv_name)
def plot_graph_best_score_gender_combination_from_tomato_quentin (df):

    df_sort_public_score = df.sort_values(by='tomatoMetermoyen', ascending=False)

    # Réinitialiser les index après le tri (optionnel)
    df_sort_public_score = df_sort_public_score.reset_index(drop=True)
    # Supprimer les lignes où 'audienceScoremoyenne' ou 'tomatoMetermoyen' est NaN
    df_sort_public_score = df_sort_public_score.dropna(subset=['audienceScoremoyenne', 'tomatoMetermoyen'])
    df_sort_public_score = df_sort_public_score.reset_index(drop=True)

    genres_to_remove = ['Supernatural', 'Medical' ]

    # Fonction pour vérifier si l'un des genres à supprimer est dans la combinaison
    def contains_genre(combo, genres_to_remove):
        return any(genre in combo for genre in genres_to_remove)

    # Supprimer les lignes contenant les genres de la liste dans 'combinations'
    df_sort_public_score = df_sort_public_score[~df_sort_public_score['combinations'].apply(lambda x: contains_genre(x, genres_to_remove))]

    df_sort_public_score = df_sort_public_score.reset_index(drop=True)
    # Afficher le DataFrame nettoyé
    df_sort_public_score.head(20)

        # Étape 1 : Nettoyer les genres et extraire tous les genres uniques dans les combinaisons
    def clean_genres(genre_string):
        # Nettoie les caractères indésirables comme les guillemets et les parenthèses
        return [re.sub(r"[\'\(\)\[\]]", "", genre).strip() for genre in genre_string.split(', ')]

    # Extraire et nettoyer les genres uniques
    genres = []
    for combo in df_sort_public_score['combinations']:
        combo_clean = clean_genres(combo)
        genre1 = combo_clean [0]
        genre2 = combo_clean [1]

        if genre1 not in genres:
            genres.append(genre1)
        if genre2 not in genres:
            genres.append(genre2)

        if len(genres) > N:
            break


    top_20_genres = genres[:N]  # Limiter à 20 genres (ou tout autre nombre N)


    # Étape 2 : Créer une matrice vide pour les genres
    genre_matrix = pd.DataFrame(0, index=top_20_genres, columns=top_20_genres)

    # Étape 3 : Remplir la matrice avec les fréquences des combinaisons
    for combo, count in zip(df['combinations'], df['tomatoMetermoyen']):
        genres_in_combo = clean_genres(combo)
        
        # Ne conserver que les genres qui figurent dans le top_20_genres
        genres_in_combo = [genre for genre in genres_in_combo if genre in top_20_genres]

        # Vérifier que genres_in_combo contient plus d'un genre, sinon ignorer cette combinaison
        if len(genres_in_combo) > 1:
            # Créer une matrice symétrique des combinaisons (genre1, genre2)
            for i in range(len(genres_in_combo)):
                for j in range(i + 1, len(genres_in_combo)):
                    genre1, genre2 = sorted([genres_in_combo[i], genres_in_combo[j]])

                    # Vérifier si la combinaison (genre1, genre2) existe, sinon (genre2, genre1)
                    if genre_matrix.loc[genre1, genre2] == 0:
                        genre_matrix.loc[genre1, genre2] = count
                    else:
                        genre_matrix.loc[genre2, genre1] = count

                    # Assurer la symétrie (genre1, genre2) == (genre2, genre1)
                    genre_matrix.loc[genre2, genre1] = genre_matrix.loc[genre1, genre2]  # Assurer la symétrie


        # Remplacer les NaN par 0 pour les afficher en blanc
    genre_matrix_filled = genre_matrix.fillna(0)

    # Créer un masque où les valeurs sont égales à 0
    zero_mask = genre_matrix_filled == 0

    # Trouver le minimum des valeurs non nulles
    arr = genre_matrix.values[~np.isnan(genre_matrix.values)]
    vminim = np.min(arr[arr > 0])  # Minimum non nul
    vminim = int(vminim)  # Convertir en entier pour l'affichage
    vmaxim = int(np.nanmax(arr))  # Maximum des valeurs

    print(f"Minimum (non nul) pour LogNorm: {vminim}")
    print(f"Maximum pour LogNorm: {vmaxim}")

    # Créer la heatmap avec une échelle logarithmique
    plt.figure(figsize=(12, 10))

    ax = sns.heatmap(
        genre_matrix_filled, 
        cmap='plasma', 
        annot=True, 
        fmt=".0f", 
        cbar_kws={'label': 'Occurrences'},
        norm=LogNorm(vmin=vminim, vmax=vmaxim),  # Appliquer LogNorm
        mask=zero_mask,  # Masquer les 0
        annot_kws={'size': 8}
    )

    # Récupérer la colorbar associée
    cbar = ax.collections[0].colorbar

    # Appliquer un format personnalisé pour afficher les ticks en entier
    cbar.ax.yaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{int(x)}"))

    # Ajouter un titre et ajuster les étiquettes
    plt.title('Heatmap des meilleures combinaisons de sous-genres (tomato score)')
    plt.xticks(rotation=90, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()

    # Sauvegarder l'image
    path_plots_save = "./LesPlots/"  # Remplacez par le chemin où vous voulez sauvegarder
    plt.savefig(path_plots_save + "heatmap_best_tomato_Metermoyen_scoreby_subgenre_combination.png", dpi=300, bbox_inches='tight')

    # Afficher la heatmap
    plt.show()
